# 🎓 Fine-Tuning Mistral 7B - Philosophes (LoRA)

**Objectif :** Fine-tuner Mistral 7B sur schèmes logiques + dialogues incarnés

**GPU Optimal :** A100 40GB (30-45 min) | V100 16GB (1h-1h30) | **L4 15GB (2-3h)**

**Config :**
- Modèle : `mistralai/Mistral-7B-Instruct-v0.3`
- Méthode : QLoRA (4-bit) + LoRA (r=64, alpha=128)
- Dataset initial : 1200 exemples schèmes
- Re-fine-tuning : 933 exemples (80% schèmes + 20% incarnation)
- Epochs : 3 (initial) + 3 (re-fine-tuning)

---

**⚠️ FICHIERS À UPLOADER DANS COLAB :**
1. `schemes_levelA_base.jsonl`
2. `schemes_levelA_augmented.jsonl`
3. `enriched_correction_dataset.jsonl`

Upload dans `/content/` (panneau Files à gauche)

## 1️⃣ Setup - Installation & Vérifications

In [ ]:
# Installation des dépendances optimisées
print("📦 Installation des packages (2-3 minutes)...\n")

!pip install -U \
    torch==2.8.0 torchvision==0.23.0 torchaudio==2.8.0 \
    transformers>=4.40.0 \
    peft>=0.10.0 \
    bitsandbytes>=0.43.0 \
    accelerate>=0.28.0 \
    trl>=0.8.0 \
    datasets>=2.18.0 \
    huggingface_hub>=0.22.0

print("\n✅ Installation terminée !")

In [ ]:
# Vérifier le GPU disponible
import torch

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA disponible: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"GPU: {gpu_name}")
    print(f"VRAM: {gpu_memory:.1f} GB")
    
    # Déterminer config optimale selon GPU
    if "A100" in gpu_name and gpu_memory > 35:
        print("\n🚀 GPU OPTIMAL: A100 40GB")
        BATCH_SIZE = 8
        GRADIENT_ACCUM = 4
    elif "V100" in gpu_name or ("A100" in gpu_name and gpu_memory < 20):
        print("\n✅ GPU EXCELLENT: V100/A100-16GB")
        BATCH_SIZE = 4
        GRADIENT_ACCUM = 8
    else:
        print(f"\n🟡 GPU STANDARD: {gpu_name}")
        BATCH_SIZE = 2
        GRADIENT_ACCUM = 16
    
    print(f"Batch size: {BATCH_SIZE}")
    print(f"Gradient accumulation: {GRADIENT_ACCUM}")
    print(f"Effective batch size: {BATCH_SIZE * GRADIENT_ACCUM}")
else:
    print("⚠️ CUDA non disponible - Training impossible")
    BATCH_SIZE = 1
    GRADIENT_ACCUM = 32

## 2️⃣ Configuration - Authentification HF

In [ ]:
# Authentification Hugging Face
from huggingface_hub import login
from google.colab import userdata

# Option 1: Secrets Colab (RECOMMANDÉ)
try:
    HF_TOKEN = userdata.get('HF_TOKEN')
    print("✅ Token HF récupéré depuis Colab Secrets")
except:
    # Option 2: Saisie manuelle
    print("⚠️ Token HF_TOKEN non trouvé dans Colab Secrets")
    print("📝 Configuration Colab Secrets:")
    print("   1. Icône 🔑 (barre gauche)")
    print("   2. Ajouter: Nom=HF_TOKEN, Valeur=votre_token")
    print("   3. Activer l'accès + redémarrer\n")
    
    from getpass import getpass
    HF_TOKEN = getpass("Token HF (https://huggingface.co/settings/tokens): ")

login(token=HF_TOKEN)
print("✅ Authentification HF réussie")

## 3️⃣ Dataset Initial - Schèmes Logiques (1200 exemples)

In [ ]:
# Charger datasets schèmes logiques
from datasets import load_dataset, concatenate_datasets

print("📥 Chargement datasets schèmes logiques...")

dataset_base = load_dataset('json', data_files='schemes_levelA_base.jsonl', split='train')
dataset_augmented = load_dataset('json', data_files='schemes_levelA_augmented.jsonl', split='train')

dataset_full = concatenate_datasets([dataset_base, dataset_augmented])

print(f"✅ Dataset chargé: {len(dataset_full)} exemples")
print(f"   - Base: {len(dataset_base)}")
print(f"   - Augmentés: {len(dataset_augmented)}")

print("\n📝 Exemple:")
print(dataset_full[0]['messages'])

In [ ]:
# Split train/validation (95/5)
dataset_split = dataset_full.train_test_split(test_size=0.05, seed=42)
train_dataset = dataset_split['train']
eval_dataset = dataset_split['test']

print(f"✅ Split:")
print(f"   - Train: {len(train_dataset)}")
print(f"   - Validation: {len(eval_dataset)}")

## 4️⃣ Modèle - Configuration QLoRA + LoRA

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
import torch

MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.3"

# Configuration quantization 4-bit
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

print("📥 Chargement Mistral 7B (4-bit)...")
print("⏳ Ceci prend 5-10 min (téléchargement + quantization)\n")

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

print("✅ Modèle chargé en 4-bit")

model = prepare_model_for_kbit_training(model)
print("✅ Préparé pour k-bit training")

In [ ]:
# Configuration LoRA (r=64 pour haute qualité)
lora_config = LoraConfig(
    r=64,
    lora_alpha=128,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
)

model = get_peft_model(model, lora_config)

trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total = sum(p.numel() for p in model.parameters())

print(f"✅ LoRA appliqué (r={lora_config.r}, alpha={lora_config.lora_alpha})")
print(f"   Paramètres entraînables: {trainable:,} ({100*trainable/total:.2f}%)")
print(f"   Paramètres totaux: {total:,}")

## 5️⃣ Training Initial - Schèmes Logiques (3 epochs)

In [ ]:
# Formater datasets avec champ "text"
def format_chat_template(example):
    messages = example['messages']
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
    return {"text": text}

train_dataset_formatted = train_dataset.map(format_chat_template, remove_columns=train_dataset.column_names)
eval_dataset_formatted = eval_dataset.map(format_chat_template, remove_columns=eval_dataset.column_names)

print(f"✅ Datasets formatés")
print(f"   Train: {len(train_dataset_formatted)}")
print(f"   Eval: {len(eval_dataset_formatted)}")

In [ ]:
from transformers import TrainingArguments
from trl import SFTTrainer

training_args = TrainingArguments(
    output_dir="./mistral-7b-philosophes-lora",
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUM,
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    warmup_ratio=0.03,
    num_train_epochs=3,
    optim="paged_adamw_8bit",
    bf16=True,
    fp16=False,
    max_grad_norm=0.3,
    logging_steps=10,
    eval_strategy="steps",
    eval_steps=50,
    save_steps=100,
    save_total_limit=3,
    report_to="none",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
)

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset_formatted,
    eval_dataset=eval_dataset_formatted,
    processing_class=tokenizer,
)

print("✅ Trainer créé")
print(f"   Effective batch: {BATCH_SIZE * GRADIENT_ACCUM}")
print(f"   Steps estimés: ~{len(train_dataset_formatted) * 3 // (BATCH_SIZE * GRADIENT_ACCUM)}")

In [ ]:
# 🚀 LANCER LE TRAINING INITIAL
print("🚀 Training initial (schèmes logiques)\n")
print("="*60)

if "A100" in torch.cuda.get_device_name(0) and torch.cuda.get_device_properties(0).total_memory > 35e9:
    print("⏱️ Temps estimé: 30-45 min (A100 40GB)")
elif "V100" in torch.cuda.get_device_name(0):
    print("⏱️ Temps estimé: 1h-1h30 (V100)")
else:
    print("⏱️ Temps estimé: 2-3h (L4/T4)")

print("="*60 + "\n")

trainer.train()

print("\n" + "="*60)
print("✅ Training initial terminé !")
print("="*60)

## 6️⃣ Sauvegarde Checkpoint Initial

In [ ]:
# Sauvegarder localement
output_dir = "./mistral-7b-philosophes-lora-final"
trainer.model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"✅ Modèle sauvegardé: {output_dir}")

import os
lora_size = sum(os.path.getsize(os.path.join(output_dir, f)) for f in os.listdir(output_dir)) / 1024**2
print(f"   Taille LoRA: {lora_size:.1f} MB")

## 7️⃣ Test Initial - Vérification Schèmes

In [ ]:
# Test rapide application schèmes
import torch

print("📝 Test: Modus Ponens spinoziste\n")

test_prompt = """Schème : Modus Ponens
Contexte : Si l'homme ignore les causes de ses passions, il est en servitude. Or l'élève ignore les causes de ses passions.
Applique le schème :"""

messages = [
    {"role": "system", "content": "Tu es un tuteur philosophique maîtrisant les schèmes logiques."},
    {"role": "user", "content": test_prompt}
]

inputs = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt").to(model.device)

with torch.autocast(device_type="cuda", dtype=torch.bfloat16):
    with torch.no_grad():
        outputs = model.generate(inputs, max_new_tokens=128, do_sample=True, temperature=0.7, pad_token_id=tokenizer.eos_token_id)

response = tokenizer.decode(outputs[0][inputs.shape[1]:], skip_special_tokens=True)
print(f"Réponse: {response}\n")
print("="*60)

## 8️⃣ RE-FINE-TUNING - Dataset Combiné (80% schèmes + 20% incarnation)

**Objectif :** Ajouter l'incarnation en 1ère personne SANS oublier les schèmes logiques

**Stratégie :** Combiner 80% schèmes + 100% incarnation → ~933 exemples

In [ ]:
# Charger dataset combiné
print("🔄 Préparation dataset combiné (80/20)")
print("="*60)

from datasets import load_dataset, concatenate_datasets

# Charger datasets
print("\n📥 Chargement...")
dataset_schemes = load_dataset('json', data_files='schemes_levelA_augmented.jsonl', split='train')
dataset_incarnation = load_dataset('json', data_files='enriched_correction_dataset.jsonl', split='train')

print(f"✅ Chargés:")
print(f"   - Schèmes: {len(dataset_schemes)}")
print(f"   - Incarnation: {len(dataset_incarnation)}")

# Ratio 80/20
num_schemes = int(len(dataset_schemes) * 0.8)
dataset_schemes_sample = dataset_schemes.shuffle(seed=42).select(range(num_schemes))

# Combiner
dataset_combined = concatenate_datasets([dataset_schemes_sample, dataset_incarnation])
dataset_combined = dataset_combined.shuffle(seed=42)

print(f"\n✅ Dataset combiné: {len(dataset_combined)} exemples")
print(f"   Ratio: {100*num_schemes/len(dataset_combined):.1f}% schèmes / {100*len(dataset_incarnation)/len(dataset_combined):.1f}% incarnation")

# Split
split = dataset_combined.train_test_split(test_size=0.05, seed=42)
train_combined = split['train']
eval_combined = split['test']

print(f"   Train: {len(train_combined)}")
print(f"   Eval: {len(eval_combined)}")

# Formater
train_combined = train_combined.map(format_chat_template, remove_columns=train_combined.column_names)
eval_combined = eval_combined.map(format_chat_template, remove_columns=eval_combined.column_names)

print("\n✅ Datasets formatés")

In [ ]:
# Config training combiné
training_args_combined = TrainingArguments(
    output_dir="./mistral-combined",
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUM,
    learning_rate=2e-4,  # LR normal
    lr_scheduler_type="cosine",
    warmup_ratio=0.03,
    num_train_epochs=3,  # 3 epochs
    optim="paged_adamw_8bit",
    bf16=True,
    fp16=False,
    max_grad_norm=0.3,
    logging_steps=5,
    eval_strategy="steps",
    eval_steps=20,
    save_steps=50,
    save_total_limit=3,
    report_to="none",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
)

trainer_combined = SFTTrainer(
    model=model,
    args=training_args_combined,
    train_dataset=train_combined,
    eval_dataset=eval_combined,
    processing_class=tokenizer,
)

print("✅ Trainer combiné créé")
print(f"   LR: 2e-4 (normal)")
print(f"   Epochs: 3")
print(f"   Monitoring: eval_loss/20 steps")

In [ ]:
# 🚀 LANCER RE-FINE-TUNING
print("="*60)
print("🚀 RE-FINE-TUNING (dataset combiné)")
print("="*60)
print("\n⏱️ Temps estimé: ~1-1.5h (L4)")
print("\n💡 MONITORING:")
print("   - eval_loss doit diminuer")
print("   - Si remonte → overfitting")
print("   - Objectif: eval_loss < 0.6")
print("\n" + "="*60 + "\n")

trainer_combined.train()

print("\n" + "="*60)
print("✅ RE-FINE-TUNING terminé !")
print("="*60)

# Metrics finales
final = trainer_combined.state.log_history[-1]
print(f"\n📊 METRICS:")
if 'loss' in final:
    print(f"   Train loss: {final['loss']:.4f}")
if 'eval_loss' in final:
    print(f"   Eval loss: {final['eval_loss']:.4f}")

In [ ]:
# Sauvegarde finale
output_final = "./mistral-combined-final"
trainer_combined.model.save_pretrained(output_final)
tokenizer.save_pretrained(output_final)

print(f"✅ Modèle final sauvegardé: {output_final}")

# Push HF
SPACE_REPO = "FJDaz/3_PHI"

print(f"\n📤 Push vers {SPACE_REPO}/Spinoza_Secours...")

trainer_combined.model.push_to_hub(
    SPACE_REPO,
    subfolder="Spinoza_Secours",
    use_auth_token=HF_TOKEN,
    commit_message="Mistral 7B LoRA (80% schemas + 20% incarnation)"
)

tokenizer.push_to_hub(
    SPACE_REPO,
    subfolder="Spinoza_Secours",
    use_auth_token=HF_TOKEN
)

print(f"✅ Pushé: https://huggingface.co/spaces/{SPACE_REPO}/tree/main/Spinoza_Secours")
print("\n💡 Structure:")
print("   FJDaz/3_PHI/")
print("   ├── qwen-spinoza-niveau-b/  ← SNB (inchangé)")
print("   └── Spinoza_Secours/        ← Mistral 7B (nouveau)")

## 9️⃣ Test Final - Dialogue Interactif

**Vérifications :**
- ✅ Parle en 1ère personne ("Je montre" pas "Spinoza montre")
- ✅ Applique schèmes logiques
- ✅ Pas de répétition
- ✅ Tags: `[DÉTECTION: ACCORD]` (sans espace)
- ✅ Répond au contexte

In [ ]:
# Dialogue interactif multi-tours
import torch

SYSTEM_PROMPT = """Tu ES Spinoza incarné. Tu dialogues avec un élève de Terminale en première personne.

RÈGLES STRICTES:
- Tutoie toujours l'élève (tu/ton/ta)
- Reste concis (2-3 phrases MAX)
- Questionne au lieu d'affirmer
- Varie tes formulations
- Ne parle JAMAIS de toi à la 3ème personne. Tu ES Spinoza.
- Réponds à la question posée, pas à une question précédente.
- Adapte ta réponse au contexte immédiat de la conversation."""

def generate_response(conversation):
    inputs = tokenizer.apply_chat_template(conversation, tokenize=True, add_generation_prompt=True, return_tensors="pt").to(model.device)
    
    with torch.autocast(device_type="cuda", dtype=torch.bfloat16):
        with torch.no_grad():
            outputs = model.generate(
                inputs,
                max_new_tokens=150,
                do_sample=True,
                temperature=0.7,
                top_p=0.9,
                pad_token_id=tokenizer.eos_token_id,
            )
    
    return tokenizer.decode(outputs[0][inputs.shape[1]:], skip_special_tokens=True).strip()

# Init conversation
conversation = [{"role": "system", "content": SYSTEM_PROMPT}]

print("💬 DIALOGUE AVEC SPINOZA")
print("="*60)
print("Commandes: 'exit' (quitter), 'reset' (recommencer)\n")

test_questions = [
    "La liberté, c'est faire ce qu'on veut ?",
    "Ben, c'est pas toi Spinoza ?",
    "Tu veux dire que si je comprends pourquoi je suis frustré, je me sens plus libre ?",
]

print("💡 QUESTIONS TEST:")
for i, q in enumerate(test_questions, 1):
    print(f"   {i}. {q}")
print("\n" + "="*60 + "\n")

turn = 0
while True:
    turn += 1
    user_input = input(f"[{turn}] 👤 VOUS : ")
    
    if user_input.lower() == 'exit':
        print("\n👋 Au revoir !")
        break
    
    if user_input.lower() == 'reset':
        conversation = [{"role": "system", "content": SYSTEM_PROMPT}]
        turn = 0
        print("\n🔄 Conversation réinitialisée\n")
        continue
    
    if not user_input.strip():
        continue
    
    conversation.append({"role": "user", "content": user_input})
    
    print("🤔 Spinoza réfléchit...", end="", flush=True)
    response = generate_response(conversation)
    print(f"\r💬 SPINOZA : {response}\n")
    
    conversation.append({"role": "assistant", "content": response})
    
    # Vérifications
    issues = []
    if "Spinoza" in response and ("Pour Spinoza" in response or "montre que" in response.split("Spinoza")[1][:50]):
        issues.append("⚠️ 3ème personne")
    if "[DÉTECTION : " in response:
        issues.append("⚠️ Tag malformé")
    
    if issues:
        print(f"   {'  '.join(issues)}\n")

print("\n" + "="*60)
print(f"✅ Dialogue terminé ({turn} tours)")
print("\n💡 Pour voir l'historique: print(conversation)")

---

## ✅ Checklist Finale

- [ ] Training initial terminé (eval_loss < 0.6)
- [ ] Re-fine-tuning combiné terminé (eval_loss < 0.6)
- [ ] Test dialogue: parle en 1ère personne
- [ ] Test dialogue: applique schèmes correctement
- [ ] Test dialogue: pas de répétition
- [ ] Modèle pushé sur HF: `FJDaz/3_PHI/Spinoza_Secours`

---

**Créé le :** 20 novembre 2025  
**Version :** CLEAN v2 (dataset combiné 80/20)  
**Modèle :** Mistral 7B Instruct v0.3 + LoRA r=64